# The$\ \beta$-Hermite (Gaussian) Ensembles

Referencing paper : <i>Matrix Models for Beta Ensembles</i>, Ioana Dumitriu and Alan Edelman

- Generation of $\ \beta$-Hermite Gaussian Ensemble as in the paper
- Plot the DPP created by the matrices of this ensemble
- Finfd a dynamic version, a Dyson Brownian motion version.

## 1. Static

In [1]:
%pylab inline
from matplotlib import gridspec
plt.style.use('classic')
plt.rc('figure',facecolor='w')
plt.rc('font', size=10)
from numpy.matlib import repmat
import plotly.graph_objs as go
import plotly.offline
import sys
sys.path.append('../dyson_brownian_motion/')
from plot_tools import adjust_spines, sc_law, plot_dpp, plot_traj_with_histo, plot_traj_with_histo_sclaw


Populating the interactive namespace from numpy and matplotlib


In [2]:
def beta_Hermite_Gaussian_ensemble(dim, beta):
    diag = np.diag(np.random.randn(dim))
    u_diag_vect = np.zeros(dim-1)
    for i in range(0,dim-1):
        u_diag_vect[i] = np.sqrt(np.random.chisquare((dim-(i+1))*beta))
    u_diag = np.diag(u_diag_vect,1)
    H_b = diag + u_diag + u_diag.T
    return H_b

In [3]:
# Eigenvalues sample
N = 200
eigenvalues_samples = np.linalg.eigvalsh(beta_Hermite_Gaussian_ensemble(N, 2))

In [4]:
#plot_dpp(eigenvalues_samples, './plot/beta_hermite_eigenvalues.png')

## 2. Dynamic (classic)

In [5]:
class beta_hermite_dpp:
    def __init__(self, n_traj, n_samples, tf, beta, rescaling=False):
        self.n_traj = n_traj
        self.n_samples = n_samples
        self.tf = tf
        self.dt = tf/n_samples
        self.beta = beta
        self.brownians_list = [ [np.zeros(i*self.beta) for i in range(1, self.n_traj)] ]
        self.diag_list = [ np.zeros(self.n_traj) ]
        self.tridiag_matrices_list = [np.zeros((self.n_traj, self.n_traj))]
        self.generate()
        self.eigen_values = np.zeros((self.n_samples+1, self.n_traj))
        self.diag(rescaling)

    def generate(self):
        for sample in range(self.n_samples+1):
            
            diag = self.diag_list[sample] + np.random.randn(self.n_traj)*(self.dt)**0.5
            self.diag_list.append( diag )
            
            tridiag_matrix = np.diag( diag )
            brownians_sample_list = []
            
            for i in range(1, self.n_traj):
                brownians_sample = self.brownians_list[sample][i-1] + np.random.randn(i*self.beta)*(self.dt/2)**0.5
                brownians_sample_list.append( brownians_sample )
                
                tridiag_matrix[(self.n_traj-1)-i, (self.n_traj)-i] = np.sqrt( np.sum( brownians_sample**2 ))
                tridiag_matrix[(self.n_traj)-i, (self.n_traj-1)-i] = np.sqrt( np.sum( brownians_sample**2 ))
            
            self.brownians_list.append(brownians_sample_list)
            self.tridiag_matrices_list.append(tridiag_matrix)

    def diag(self, rescaling):
        if rescaling:
            for sample in range(1, self.n_samples+1):
                eigen_values = sorted(np.real( np.linalg.eigvalsh(self.tridiag_matrices_list[sample]) ), reverse=True)
                self.eigen_values[sample] = np.multiply(eigen_values, 1/np.sqrt((sample)*self.dt))
                #print(1/np.sqrt((sample+1)*self.dt))
        else:
            for sample in range(1, self.n_samples+1):
                self.eigen_values[sample] = \
                    sorted(np.real(np.linalg.eigvalsh(self.tridiag_matrices_list[sample])), reverse=True)

In [6]:
test_beta_hermite_dpp = beta_hermite_dpp(50, 300, 1, beta=2, rescaling=False)

In [7]:
#plot_traj_with_histo(test_beta_hermite_dpp, './plot/beta_dbm_histo.png')

## 2. Dynamic (rescale)

In [8]:
test_beta_hermite_dpp_rescale = beta_hermite_dpp(100, 300, 1, beta=2, rescaling=True)

In [9]:
#plot_traj_with_histo(test_beta_hermite_dpp_rescale, './plot/beta_dbm_rescale_histo.png')

## Holcomb - Paquette

In [10]:
from numpy.polynomial import polynomial as P
SEED = 32
np.random.seed(SEED)

In [11]:
dim =4
U=beta_Hermite_Gaussian_ensemble(dim,2)
print(U)

[[-0.34889445  2.88046957  0.          0.        ]
 [ 2.88046957  0.98370343  2.25234038  0.        ]
 [ 0.          2.25234038  0.58092283  1.37063997]
 [ 0.          0.          1.37063997  0.07028444]]


In [12]:
np.linalg.eigvals(U)

array([ 4.30919474, -3.32189999,  1.2172107 , -0.91848919])

In [13]:
lbda, vect =  np.linalg.eigh(U)
print(lbda)
print(vect)

[-3.32189999 -0.91848919  1.2172107   4.30919474]
[[ 0.61712084  0.4636926  -0.45062553 -0.448428  ]
 [-0.63694604 -0.09169229 -0.24500414 -0.72516567]
 [ 0.42837186 -0.51556862  0.55089493 -0.49719346]
 [-0.17308717  0.71468224  0.65834974 -0.16076613]]


In [14]:
np.dot(U,vect[:,0])

array([-2.05001371,  2.11587103, -1.42300847,  0.57497826])

In [16]:
lbda[0]*vect[:,0]

array([-2.05001371,  2.11587103, -1.42300847,  0.57497826])

### Définition des polynomes

In [53]:
U

array([[-0.34889445,  2.88046957,  0.        ,  0.        ],
       [ 2.88046957,  0.98370343,  2.25234038,  0.        ],
       [ 0.        ,  2.25234038,  0.58092283,  1.37063997],
       [ 0.        ,  0.        ,  1.37063997,  0.07028444]])

In [73]:
a=np.array([0]+list(np.diag(U,1))+[1])

In [74]:
b=np.diag(U,1)

array([ 2.88046957,  2.25234038,  1.37063997])

In [36]:
p_unit = (0,1)
p=[]
p.append( (1) )
p_k_1_1 = P.polymul(p_unit, p[0])
p_k_1_2 = P.polymul(U[0,0], p[0])
p_new = (1/U[0,1])* P.polysub( p_k_1_1, p_k_1_2 ) 
p.append(p_new)

In [37]:
p

[1, array([ 0.12112416,  0.34716562])]

In [38]:
for k in range(2,dim):
    print(k)
    p_k_1_1 = P.polymul(p_unit, p[k-1])
    p_k_1_2 = P.polymul(U[k-1,k-1], p[k-1])
    p_k_2 = P.polymul(U[k-1,k-2], p[k-2])
    p_new = (1/U[k-1,k])*P.polysub( P.polysub(p_k_1_1, p_k_1_2), p_k_2 ) 
    p.append(p_new)

2
3


In [39]:
p

[1,
 array([ 0.12112416,  0.34716562]),
 array([-1.33177909, -0.0978466 ,  0.1541355 ]),
 array([ 0.36541182, -1.50066608, -0.13671529,  0.11245514])]

In [40]:
P.polyval(lbda[3],p[3])

0.35851047185190504

In [41]:
0.16076613/0.448428

0.35851046321817553

In [82]:
p_n = P.polyadd(P.polymul(-U[dim-2,dim-1], p[dim-2]), P.polymul(P.polysub(p_unit, U[dim-1,dim-1]), p[dim-1]))

In [83]:
p_n

array([ 1.79970688,  0.60499776, -1.7023214 , -0.14461913,  0.11245514])

### Calcul de G

In [86]:
i=1
G = np.zeros((dim,dim))


G[0,0] = -U[0,1]*P.polyval(lbda[i],p[0])*P.polyval(lbda[i],P.polyder(p[1]))-\
            U[0,1]*P.polyval(lbda[i],p[1])*P.polyval(lbda[i],P.polyder(p[0]))

G[dim-1,dim-1] = -U[dim-2,dim-1]*P.polyval(lbda[i],p[dim-2])*P.polyval(lbda[i],P.polyder(p[dim-1]))\
            +1*P.polyval(lbda[i],p[dim-1])*P.polyval(lbda[i],P.polyder(p_n))\
            -U[dim-2,dim-1]*P.polyval(lbda[i],p[dim-1])*P.polyval(lbda[i],P.polyder(p[dim-2]))-\
            +1*P.polyval(lbda[i],p_n)*P.polyval(lbda[i],P.polyder(p[dim-1]))
            


In [91]:
for k in range(1, dim-1):
    G[k,k] = -U[k-1,k]*P.polyval(lbda[i],p[k-1])*P.polyval(lbda[i],P.polyder(p[k]))\
            +U[k,k+1]*P.polyval(lbda[i],p[k])*P.polyval(lbda[i],P.polyder(p[k+1]))\
            -U[k-1,k]*P.polyval(lbda[i],p[k])*P.polyval(lbda[i],P.polyder(p[k-1]))-\
            +U[k,k+1]*P.polyval(lbda[i],p[k+1])*P.polyval(lbda[i],P.polyder(p[k]))
            
for k in range(1,dim):
    G[k,k-1]=-U[k-1,k]*(P.polyval(lbda[i], p[k-1])*P.polyval(lbda[i], P.polyder(p[k-1]))-\
                      P.polyval(lbda[i], p[k])*P.polyval(lbda[i], P.polyder(p[k])) )

In [92]:
G

array([[-1.        ,  0.        ,  0.        ,  0.        ],
       [-0.19774371,  0.03910258,  0.        ,  0.        ],
       [ 0.        ,  1.10874523,  2.97509762,  0.        ],
       [ 0.        ,  0.        , -2.6190475 ,  3.98527682]])